In [58]:
# importing libraries
import pandas as pd 
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import joblib

In [59]:
train= pd.read_csv("train_datasetset1.csv")
train.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,44,70,1.025,0,0,0,0,0,0,121,...,48,7800,4.4,0,0,0,0,0,0,notckd
1,60,90,1.015,3,0,0,0,0,0,74,...,39,7800,4.4,1,1,0,0,1,0,ckd
2,55,80,1.025,0,0,0,0,0,0,130,...,41,9100,6.0,0,0,0,0,0,0,notckd
3,62,90,1.020,2,1,0,0,0,0,169,...,47,11000,6.1,1,0,0,0,0,0,ckd
4,25,60,1.020,0,0,0,0,0,0,119,...,40,9200,5.2,0,0,0,0,0,0,notckd


In [60]:
train['classification'].value_counts()

classification
ckd       200
notckd    120
Name: count, dtype: int64

In [61]:
from sklearn.preprocessing import LabelEncoder
for i in train.columns:
    if train[i].dtype=='object':
        train[i]=LabelEncoder().fit_transform(train[i])


In [62]:
X_train=train.drop(['classification'],axis = 1 )
y_train=train['classification']

In [63]:
test= pd.read_csv("unseen_dataset1.csv")
for i in test.columns:
    if test[i].dtype=='object':
        test[i]=LabelEncoder().fit_transform(test[i])

In [64]:
y_test=test['classification']
X_test=test.drop(['classification'],axis = 1 )

In [65]:
model_params = {
    
    'RF': {
        'model': RandomForestClassifier(),
        'params' : {
            'max_depth' : [20,10,5,7],

        }
    },
    'LR' : {
        'model': LogisticRegression(),
        'params': {
            "solver" : ['newton-cg','lbfgs','liblinear','sag','saga']

                     
        }}
    ,
    'DT': {
        'model': DecisionTreeClassifier(),
        'params': {
       
            'max_depth' : [20,10,5,7]
        }
        
    }  
    ,
   
 'SVM': {
     'model': SVC(probability=True),
    'params':  {
                'kernel': ['linear','rbf','poly',]
    }},

    
  'NB': {
     'model': GaussianNB(),
    'params':  {
      'var_smoothing': np.logspace(0,-9, num=100)
    }
      
 },
    
    
   } 

   

In [66]:
for model_name, mp in model_params.items():
    print(model_name)

RF
LR
DT
SVM
NB


In [67]:
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average='weighted'),
           'recall' : make_scorer(recall_score, average='weighted'), 
           'f1' : make_scorer(f1_score, average='weighted')}


In [ ]:
import warnings
warnings.filterwarnings('ignore')
scores = []
model={}
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=10, scoring =make_scorer(f1_score, average='weighted'),return_train_score=False)
    clf.fit(X_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
   
    model[model_name]=clf.best_estimator_
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

In [ ]:
clf = [('RF', model['RF']),('LR', model['LR']), ('DT', model['DT']), ('SVM', model['SVM']), ('NB', model['NB']), ] 
stack_model = StackingClassifier(estimators = clf,final_estimator =RandomForestClassifier())
stacking = stack_model.fit(X_train,y_train)

In [ ]:
model.update({'Stacking':stacking} )

In [ ]:
def evaluate_models(models, X_train, X_val, y_train, y_val):
 
    scores = list()
    for name, model in models:

        yhat = model.predict(X_val)
        acc = accuracy_score(y_val, yhat)
        
        scores.append(acc)
        # report model performance
    return scores
 

In [ ]:

ReultofTest=pd.DataFrame([])


for model_name, i in model.items():   
    Model=i

    AccuracyTest=[]
    PrecisionTest=[]
    RecallTest=[]
    F1Test=[]
    
    

    
    for i in range(0,1):
        if i==0:
            r=70
        elif i== 1:
            r=80
        elif i== 2:
            r=90

        SFold = StratifiedKFold(n_splits=10,  shuffle=True,random_state=r)
        Train_Score = cross_validate(Model, X_train, y_train, scoring=scoring,  cv=SFold)
  
       
        
        y_pred = Model.predict(X_test)
    
     
       
        Accuracy_Test= accuracy_score(y_test,y_pred)
        Precision_Test=precision_score(y_test, y_pred, average='weighted')
        Recall_Test=recall_score(y_test, y_pred, average='weighted')
        F1_Test=f1_score(y_test, y_pred, average='weighted') 
        
        
        
        
        AccuracyTest.append(round(100*Accurcy_Test, 2))
        PrecisionTest.append(round(100*Precision_Test, 2))
        RecallTest.append(round(100*Recall_Test, 2))
        F1Test.append(round(100*F1_Test, 2)) 
       
      
     
 
        
        finalSTD.reindex(['model','Accuracy','Precision','Recall','F1'], axis=1)
    
        ReultofTest=ReultofTest._append({'model':model_name,
                                        'AccuracyTest' : round(np.mean(AccuracyTest),2),
                                        'PrecisionTest':round(np.mean(PrecisionTest),2),
                                        'RecallTest' : round(np.mean(RecallTest),2),'F1Test':round(np.mean(F1Test),2),
                                  
                                       }, ignore_index=True)
        
        ReultofTest.reindex(['model','Accuracy','Precision','Recall','F1'],axis=1)
       

# Result of testing

In [ ]:
ReultofTest

In [ ]:
h=ReultofTest.iloc[:, 1::]
from IPython.display import HTML

HTML(h.to_html(index=False))

In [ ]:
classifiers = {'RF': model['RF'],'LR': model['LR'], 'DT': model['DT'], 
               'SVM': model['SVM'],   'NB': model['NB'], 
               
               
               'Stacking': model['Stacking']}

# Define a result table as a DataFrame
result_table = pd.DataFrame(columns=['classifiers', 'fpr','tpr','auc'])

# Train the models and record the results
for k,v in classifiers.items():
   

    
    predicted = v.predict(X_test)

    #predicted = np.argmax(predicted_proba, axis = 1)
    fpr, tpr, _ = roc_curve(y_test,  predicted)
    auc = round(roc_auc_score(y_test, predicted),5)*100
    
    result_table = result_table._append({'classifiers':k,
                                        'fpr':fpr, 
                                        'tpr':tpr, 
                                        'auc':auc}, ignore_index=True)

# Set name of the classifiers as index labels
result_table.set_index('classifiers', inplace=True)


In [ ]:
fig = plt.figure(figsize=(8,6))

for i in result_table.index:
    plt.plot(result_table.loc[i]['fpr'], 
             result_table.loc[i]['tpr'], 
             label="{}, AUC={:.3f}".format(i, result_table.loc[i]['auc']))
    
plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("Flase Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

plt.show()